<a href="https://colab.research.google.com/github/Loki0710/githubemc1/blob/main/MDAnalysis_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install MDAnalysis

In [ ]:
!pip install nglview

In [ ]:
#import tools
# MDAnalysis tools
import MDAnalysis as mda
from MDAnalysis.analysis import rms, align, pca
# To view the trajs on notebook
import nglview as nv

In [ ]:
# Data processing
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)  #Suppress scientific notation

from sklearn.decomposition import PCA

In [ ]:
# Plot & System tools
import matplotlib as mpl
import matplotlib.pyplot as plt
from  matplotlib.colors import ListedColormap, NoNorm, BoundaryNorm, CSS4_COLORS
import seaborn as sns

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

#Change working directory to current directory
import os
os.chdir(os.getcwd() )

In [ ]:
!wget https://raw.githubusercontent.com/Rajnishphe/MolecularDynamics-Tutorial/main/Trajectory_files/md.gro
!wget https://raw.githubusercontent.com/Rajnishphe/MolecularDynamics-Tutorial/main/Trajectory_files/md_center.xtc


In [ ]:
ls

In [ ]:
u = mda.Universe("md.gro","md_center.xtc")
print (u)
print (u.atoms)

In [ ]:
print('Atoms: ', u.atoms.n_atoms)
print('Residues: ', u.residues.n_residues)
print('Segments: ', u.segments.n_segments)

In [ ]:
a = u.atoms[0:50]
list (a.resids)

In [ ]:
from MDAnalysis.analysis import align, rms
aligner = align.AlignTraj(u, u, select='name CA', in_memory=True).run()

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
#make protein, ligand and complex as independent entries
protein = u.atoms.select_atoms("protein")
print (protein)

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
import MDAnalysis.analysis.rms
import matplotlib.pyplot as plt
import MDAnalysis as mda
import pandas as pd
import numpy as np

In [ ]:
!pip install numpy scikit-learn matplotlib

#Protein RMSD

In [ ]:

R = MDAnalysis.analysis.rms.RMSD(protein, protein,
           select="backbone", ref_frame=1)
R.run()
rmsd = R.rmsd.T   # transpose makes it easier for plotting
time = rmsd[1]
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.plot(time, rmsd[2], 'k-',  label="all")
ax.legend(loc="best")
ax.set_xlabel("time (ps)")
ax.set_ylabel(r"RMSD ($\AA$)")
fig.savefig("rmsd_all_CORE_LID_NMP_ref1AKE.pdf")

In [ ]:
# Create dataframe of RMSD values
# RMSD routine creates Frame and Time columns + three groups of RMSD calculations
cols = ['Frame','Time', 'Full']
df_rms = pd.DataFrame(R.rmsd, columns=cols)

In [ ]:
# Lets convert time to ns
df_rms.Time/1000.0 #  Time*(1/1000)

In [ ]:
df_rms.drop('Frame',axis=1,inplace=True)      #Drop Frame column, we retain only Time
df_rms['Time' ] = df_rms.Time* 0.001          #Convert time to ns
df_rms.set_index('Time',inplace=True)         #Set Time as index (row labels)

In [ ]:
#Save rmsd output to csv file
#df_rms.to_csv( fout_rms ,index_label='Time')

display( df_rms.head(5), df_rms.tail(4) )

In [ ]:
# Lets do time evolution of RMSD
df_rms.plot()
plt.xlim( [0,1])
plt.xlabel('Time (ns)')
plt.ylabel( ' RMSD $\AA$')

plt.legend(ncol=4,fontsize=9,loc=3)
#plt.legend(ncol=4,fontsize=9,loc=3)
plt.savefig('Protein_RMSD.tif')

#Protein RMSF Calculation

In [ ]:
c_alphas = u.select_atoms('protein and name CA')
R = rms.RMSF(c_alphas).run()
%matplotlib inline

In [ ]:
plt.plot(c_alphas.resids, R.rmsf, )
plt.xlim(0,150)
plt.xlabel('Residue Number', fontsize=18)
plt.ylabel('RMSF ($\AA$)', fontsize=18)
mpl.rcParams.update({'font.size': 14})
#plt.legend(loc='lower center')
plt.title('Protein RMSF')
plt.savefig('Protein_RMSF.png')

In [ ]:
#PCA Analysis https://userguide.mdanalysis.org/1.0.0/examples/analysis/reduced_dimensions/pca.html
aligner = align.AlignTraj(u, u, select='backbone', in_memory=True).run() # align the trajectory first

In [ ]:
from MDAnalysis.analysis import pca, align
pc = pca.PCA(u, select='backbone', align=False, mean=None, n_components=None).run()

In [ ]:
pc.variance[0]

In [ ]:
print(pc.cumulated_variance[0])
print(pc.cumulated_variance[5])

In [ ]:
plt.plot(pc.cumulated_variance[:50])
plt.xlabel('Principal component')
plt.ylabel('Cumulative variance');

In [ ]:
backbone = u.select_atoms('backbone')
n_bb = len(backbone)
print('There are {} backbone atoms in the analysis'.format(n_bb))
print(pc.p_components.shape)

In [ ]:
transformed = pc.transform(backbone, n_components=5) # output only 5 PCs
transformed.shape

In [ ]:
# make a dataframe out of array
import pandas as pd
df = pd.DataFrame(transformed,
                  columns=['PC{}'.format(i+1) for i in range(5)])
df['Time (ps)'] = df.index * u.trajectory.dt
df.head()

In [ ]:
import seaborn as sns

g = sns.PairGrid(df, hue='Time (ps)',
                 palette=sns.color_palette('Oranges_d',
                                          n_colors=len(df)))
g.map(plt.scatter, marker='.')

#Pairwise RMSD calculation

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import diffusionmap, align, rms
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
matrix = diffusionmap.DistanceMatrix(u, select='name CA').run()

In [ ]:
matrix.dist_matrix.shape

In [ ]:
plt.imshow(matrix.dist_matrix, cmap='viridis')
plt.xlabel('Frame')
plt.ylabel('Frame')
plt.colorbar(label=r'RMSD ($\AA$)')

#Radius of gyration

In [ ]:
len(u.trajectory)

In [ ]:
for ts in u.trajectory:
    print("Frame: {0:5d}, Time: {1:8.3f} ps".format(ts.frame, u.trajectory.time))
    print("Rgyr: {0:g} A".format(u.atoms.radius_of_gyration()))

In [ ]:
Rgyr = []
protein = u.select_atoms("protein")
for ts in u.trajectory:
   Rgyr.append((u.trajectory.time, protein.radius_of_gyration()))
Rgyr = np.array(Rgyr)

In [ ]:
#Plot
import matplotlib.pyplot as plt
ax = plt.subplot(111)
ax.plot(Rgyr[:,0], Rgyr[:,1], 'r--', lw=2, label=r"$R_G$")
ax.set_xlabel("time (ps)")
ax.set_ylabel(r"radius of gyration $R_G$ ($\AA$)")
ax.figure.savefig("Rgyr.pdf")
plt.draw()